In [4]:
import pandas as pd

n_campi = 'CJBG'

levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CJBG.xlsx')
suap = pd.read_excel('consolidacao/suap/2023-08-08_16-21_CJBG_relatorio_almox.xlsx')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]
# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

print(levantamento.info())
print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código', 'Retificação'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})
# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
display(suap_cross)
display(levantamento_cross)

consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Carimbo de data/hora                  227 non-null    datetime64[ns]
 1   Endereço de e-mail                    227 non-null    object        
 2   Retificação                           3 non-null      object        
 3   Código                                227 non-null    object        
 4   Nome                                  225 non-null    object        
 5   Quantidade                            227 non-null    int64         
 6   Campus                                88 non-null     object        
 7   Localização                           43 non-null     object        
 8   Valor unitário                        39 non-null     float64       
 9   Sem identificação - SI                0 non-null      float64       
 10  Ma

,codigo,descricao_suap,und,qtd_suap,ed
0,30.07.847,"CAFÉ, APRESENTAÇÃO TORRADO E MOÍDO, CARACTERÍS...",UN,17,07 - GENEROS DE ALIMENTACAO
1,30.16.1026,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",UN,5,16 - MATERIAL DE EXPEDIENTE
2,30.16.1093,"BLOCO AUTO-COLANTE PARA RECADO, COR NATURAL, D...",UN,4,16 - MATERIAL DE EXPEDIENTE
3,30.16.1096,"COLCHETE (BAILARINA), Nº 15, COMPRIMENTO DA HA...",CAIXA,4,16 - MATERIAL DE EXPEDIENTE
4,30.16.1144,"PASTA SUSPENSA, MATERIAL CARTÃO MARMORIZADO PL...",UN,1364,16 - MATERIAL DE EXPEDIENTE
...,...,...,...,...,...
220,30.42.1160,"TORNEIRA, Material corpo metal cromado, tipo l...",UN,29,42 - FERRAMENTAS
221,30.44.41,Círculo ou seta indicativo de H2O. Cor interna...,UN,5,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
222,30.44.42,Círculo ou seta indicativo de CO2. Cor interna...,UN,16,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
223,30.44.43,Círculo ou seta indicativo de PQS. Cor interna...,UN,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS


,codigo,descricao_lev,qtd_lev,servidor
0,30.16.2565,"PASTA ARQUIVO, MATERIAL CART",56,cmpsg@jaboatao.ifpe.edu.br
1,30.16.2551,CANETA ESFEROGR,13,cmpsg@jaboatao.ifpe.edu.br
2,30.16.2554,"MINA GRAFITE, MATERIAL GRAFITA, DI",6,cmpsg@jaboatao.ifpe.edu.br
3,30.16.2560,"MINA GRAFITE, MATERIAL GRAFITA, DI",7,cmpsg@jaboatao.ifpe.edu.br
4,30.16.2057,"TINTA PARA CARIMBO, COR PRETA, COMPONENTES BASE D",10,cmpsg@jaboatao.ifpe.edu.br
...,...,...,...,...
222,30.17.965,"TECLADO MICROCOMPUTADOR, TIPO NUM",2,cmpsg@jaboatao.ifpe.edu.br
223,30.17.571,"CABO EXTENSOR, FLEX",31,cmpsg@jaboatao.ifpe.edu.br
224,30.17.398,"MOUSE PAD, APOIO DE PUNHOS ERGONOMICO, BASE PA...",25,ceof@jaboatao.ifpe.edu.br
225,30.26.1646,"PARAFUSO, MATERIAL AÇO, 6MM, CABEÇA TIPO PANEL...",80,cmpsg@jaboatao.ifpe.edu.br


In [5]:
consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.07.847,"CAFÉ, APRESENTAÇÃO TORRADO E MOÍDO, CARACTERÍS...",UN,17,07 - GENEROS DE ALIMENTACAO,CAF,17,cmpsg@jaboatao.ifpe.edu.br,both,0
1,30.16.1026,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",UN,5,16 - MATERIAL DE EXPEDIENTE,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",5,claudeandela.cavalcanti@reitoria.ifpe.edu.br,both,0
2,30.16.1093,"BLOCO AUTO-COLANTE PARA RECADO, COR NATURAL, D...",UN,4,16 - MATERIAL DE EXPEDIENTE,"BLOCO AUTO-COLANTE PARA RECADO, COR NATURAL, D...",4,cmpsg@jaboatao.ifpe.edu.br,both,0
3,30.16.1096,"COLCHETE (BAILARINA), Nº 15, COMPRIMENTO DA HA...",CAIXA,4,16 - MATERIAL DE EXPEDIENTE,"COLCHETE (BAILARINA), N",4,edmar.filho@reitoria.ifpe.edu.br,both,0
4,30.16.1144,"PASTA SUSPENSA, MATERIAL CARTÃO MARMORIZADO PL...",UN,1364,16 - MATERIAL DE EXPEDIENTE,"PASTA SUSPENSA, MATERIAL CART",1364,cmpsg@jaboatao.ifpe.edu.br,both,0
...,...,...,...,...,...,...,...,...,...,...
222,30.42.1160,"TORNEIRA, Material corpo metal cromado, tipo l...",UN,29,42 - FERRAMENTAS,"TORNEIRA, Material corpo metal cromado, tipo l...",29,ceof@jaboatao.ifpe.edu.br,both,0
223,30.44.41,Círculo ou seta indicativo de H2O. Cor interna...,UN,5,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de H2O. Cor interna...,5,ceof@jaboatao.ifpe.edu.br,both,0
224,30.44.42,Círculo ou seta indicativo de CO2. Cor interna...,UN,16,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de CO2. Cor interna...,16,ceof@jaboatao.ifpe.edu.br,both,0
225,30.44.43,Círculo ou seta indicativo de PQS. Cor interna...,UN,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de PQS. Cor interna...,2,ceof@jaboatao.ifpe.edu.br,both,0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.07.847,07 - GENEROS DE ALIMENTACAO,"CAFÉ, APRESENTAÇÃO TORRADO E MOÍDO, CARACTERÍS...",17,17,0,adequado
1,30.16.1026,16 - MATERIAL DE EXPEDIENTE,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",5,5,0,adequado
2,30.16.1093,16 - MATERIAL DE EXPEDIENTE,"BLOCO AUTO-COLANTE PARA RECADO, COR NATURAL, D...",4,4,0,adequado
3,30.16.1096,16 - MATERIAL DE EXPEDIENTE,"COLCHETE (BAILARINA), Nº 15, COMPRIMENTO DA HA...",4,4,0,adequado
4,30.16.1144,16 - MATERIAL DE EXPEDIENTE,"PASTA SUSPENSA, MATERIAL CARTÃO MARMORIZADO PL...",1364,1364,0,adequado
...,...,...,...,...,...,...,...
222,30.42.1160,42 - FERRAMENTAS,"TORNEIRA, Material corpo metal cromado, tipo l...",29,29,0,adequado
223,30.44.41,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de H2O. Cor interna...,5,5,0,adequado
224,30.44.42,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de CO2. Cor interna...,16,16,0,adequado
225,30.44.43,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de PQS. Cor interna...,2,2,0,adequado
